In [2]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [4]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('medications-use.csv');

In [4]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [10]:
SELECT NomeMedicamento, COUNT(*) Usuarios 
    FROM Medicamento, UsoMedicamento 
    WHERE Codigo = CodMedicamento GROUP BY CodMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [16]:
DROP VIEW IF EXISTS QtdadeUsados;

CREATE VIEW QtdadeUsados AS
SELECT CodMedicamento, Count(*) AS Quantidade
    FROM UsoMedicamento
    GROUP BY CodMedicamento;

SELECT *
    FROM QtdadeUsados
    WHERE Quantidade = (SELECT MAX(Quantidade) from QtdadeUsados)

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [ ]:
SELECT CodMedicamento, Count(*) AS Quantidade
    FROM UsoMedicamento
    GROUP BY CodMedicamento
    ORDER BY Quantidade DESC
    LIMIT 1;

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [9]:
SELECT NomeMedicamento, COUNT(*) QtdadeUsados
    FROM Medicamento, UsoMedicamento
    WHERE Codigo = CodMedicamento
    GROUP BY CodMedicamento
    ORDER BY QtdadeUsados DESC
    LIMIT 1;

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [10]:
SELECT AVG(QtdMedicamentos) Media
    FROM (SELECT COUNT(*) QtdMedicamentos
              FROM UsoMedicamento
              GROUP BY IdPessoa);

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [11]:
SELECT IdPessoa, COUNT(*) QtdMedicamentos
    FROM UsoMedicamento
    GROUP BY IdPessoa
    HAVING QtdMedicamentos > (SELECT AVG(QtdMedicamentos) Media
                                    FROM (SELECT COUNT(*) QtdMedicamentos
                                            FROM UsoMedicamento
                                            GROUP BY IdPessoa))

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [12]:
SELECT IdadePessoa, AVG(QtdMedicamentos) Media
    FROM (SELECT Idade IdadePessoa, COUNT(*) QtdMedicamentos
              FROM Pessoa, UsoMedicamento
              WHERE Id = IdPessoa
              GROUP BY IdPessoa)
    GROUP BY IdadePessoa
    ORDER BY IdadePessoa;

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [ ]:
CREATE VIEW UsoMedicamentoB AS
    SELECT IdPessoa IdPessoaB, CodMedicamento CodMedicamentoB
        FROM UsoMedicamento;

CREATE VIEW MedicamentosConjunto AS
    SELECT U1.CodMedicamento medicamentoA, U2.CodMedicamentoB medicamentoB
        FROM UsoMedicamento U1, UsoMedicamentoB U2
        WHERE U1.IdPessoa = U2.IdPessoaB AND U1.CodMedicamento > U2.CodMedicamentoB;

SELECT * FROM MedicamentosConjunto;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [ ]:
SELECT medicamentoA, medicamentoB, COUNT(*) QuantidadeConjunta
    FROM MedicamentosConjunto
    GROUP BY medicamentoA, medicamentoB;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [3]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [5]:
UPDATE Matriz
    SET IdadeAte60 = TRUE
    WHERE Idade <=60;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [6]:
UPDATE Matriz
    SET MaisUmAnoMedicamento = TRUE
    WHERE Id IN (SELECT DISTINCT IdPessoa
                     FROM UsoMedicamento
                     WHERE DiasUso > 365);

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [7]:
UPDATE Matriz
    SET MedicamentosAcimaMedia = TRUE
    WHERE Id IN (SELECT IdPessoa
                    FROM UsoMedicamento
                    GROUP BY IdPessoa
                    HAVING COUNT(*) > (SELECT AVG(QtdMedicamentos) Media
                                            FROM (SELECT COUNT(*) QtdMedicamentos
                                                      FROM UsoMedicamento
                                                      GROUP BY IdPessoa)));
              
SELECT * FROM Matriz WHERE MedicamentosAcimaMedia = TRUE;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

A análise proposta é descobrir se o número de idosos que utilizam medicamentos a mais de ano é maior que pessoas com menos de 60 anos  

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [31]:
CREATE VIEW Menos60 AS 
SELECT COUNT (*) QtdadeMenos60
FROM Matriz
WHERE IdadeAte60 = FALSE and MaisUmAnoMedicamento = TRUE;

CREATE VIEW Mais60 AS 
SELECT COUNT (*) QtdadeMais60
FROM Matriz
WHERE IdadeAte60 = TRUE and MaisUmAnoMedicamento = TRUE;

SELECT QtdadeMenos60, QtdadeMais60
FROM Menos60, Mais60